<a href="https://colab.research.google.com/github/elminhodev/Simulador-de-Automatos/blob/main/implementacao_mt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
import json
from typing import Dict, List

def load_spec(path: str):
    with open(path, 'r', encoding='utf-8') as f:
        spec = json.load(f)
    for k in ("initial", "final", "white", "transitions"):
        if k not in spec:
            raise ValueError(f"Spec faltando chave '{k}'")
    return spec

def load_input(path: str, white: str):
    with open(path, 'r', encoding='utf-8') as f:
        data = f.read()
    if data.endswith('\n'):
        data = data.rstrip('\n')
    if data == "":
        return white
    return data

def build_transition_map(transitions: List[Dict]):
    tmap = {}
    for t in transitions:
        key = (int(t['from']), t['read'])
        entry = (int(t['to']), t['write'], t['dir'])
        tmap.setdefault(key, []).append(entry)
    return tmap

def tape_to_str(tape: Dict[int,str], white: str) -> str:
    if not tape:
        return white
    min_i = min(tape.keys())
    max_i = max(tape.keys())
    s = ''.join(tape.get(i, white) for i in range(min_i, max_i+1))
    # corta espaços em branco das extremidades
    left, right = 0, len(s)
    while left < right and s[left] == white:
        left += 1
    while right > left and s[right-1] == white:
        right -= 1
    trimmed = s[left:right]
    return trimmed if trimmed != "" else white

def run_tm(spec_path: str, input_path: str, output_path: str = "output.txt"):
    spec = load_spec(spec_path)
    white = spec['white']
    input_str = load_input(input_path, white)

    # inicializa a fita
    tape: Dict[int,str] = {}
    for i, ch in enumerate(input_str):
        tape[i] = ch

    head = 0
    state = int(spec['initial'])
    finals = set(int(x) for x in spec['final'])
    tmap = build_transition_map(spec['transitions'])

    steps = 0
    while True:
        if state in finals:
            out_tape = tape_to_str(tape, white)
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(out_tape)
            print("1")  # aceita
            return 1

        sym = tape.get(head, white)
        key = (state, sym)
        trans_list = tmap.get(key, [])

        if not trans_list:
            out_tape = tape_to_str(tape, white)
            with open(output_path, 'w', encoding='utf-8') as f:
                f.write(out_tape)
            print("0")  # rejeita
            return 0

        to_state, write_sym, direction = trans_list[0]

        if write_sym == white:
            if head in tape and tape[head] != white:
                del tape[head]
        else:
            tape[head] = write_sym

        if direction.upper() == 'R':
            head += 1
        elif direction.upper() == 'L':
            head -= 1
        else:
            raise ValueError(f"Direção inválida: {direction}")

        state = int(to_state)
        steps += 1
        # opcional: proteger contra loops infinitos
        # if steps > 1000000:
        #     print("0")
        #     return 0


In [14]:
run_tm("/content/duplo_bal.json", "/content/duplobal.in", "/content/saida.txt")

1


1